# Graphics Idea Auto Generator

1.⁠ ⁠ask openai to generate a graphic idea description, prompt engineered for the text to diagram engine \
2.⁠ ⁠⁠auto-generate the diagram

## Using OpenAI To Generate a Graphic Idea

In [15]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import openai
from docx2python import docx2python
import re
import textwrap

In [2]:
load_dotenv()

True

In [3]:
# OpenAI API configuration
openai.api_type = "azure"
openai.api_key = os.getenv("api_key_azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("api_version")
openai_deployment = "sdgi-gpt-35-turbo-16k"


client = AzureOpenAI(
  api_key = os.getenv("api_key_azure"),  
  api_version = os.getenv("api_version"),
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

In [4]:
# use this function to make simple openAI Calls
def callOpenAI(messages):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=messages
                )
    response = response_entities.choices[0].message.content
    return response

## Text Extraction from Module Dataset

In [8]:
module_file_path = '../data/raw_data/SEA Module 3 Draft_v4.docx'
output_file = '../data/extracted_data'

In [6]:
def docx_text_cleaning(text):
    # Remove footage or image indications
    cleaned_text = re.sub(r'----[^-]+----', '', text)

    # Remove lines with only whitespace characters
    cleaned_text = cleaned_text.replace('\r', '\n')
    cleaned_text = re.sub(r"\n", " ", cleaned_text)  # remove newlines
    cleaned_text = re.sub(r"\s\s+", ' ', cleaned_text)
    cleaned_text = textwrap.fill(cleaned_text, width=80) 
    return cleaned_text

In [7]:
def extract_text_docx(file_path):
    with docx2python(file_path) as docx_content:
        raw_text = docx_content.text
        cleaned_text = docx_text_cleaning(raw_text)
        return cleaned_text

In [16]:
filename = 'SEA Module 3 Draft_v4'
content = extract_text_docx(module_file_path)
output_path = output_file + "/" + filename
with open(output_path, "w") as f:
        f.write(content)

In [17]:
def generate_graphics_idea(text):
    messages = [
            {"role": "system", "content": f"""
            You are a UNDP module writer assistant.
            Read the given passage and identify data points that would benefit from graphical representation for better understanding.
            Additionally, suggest ideas for infographics that could enhance the overall presentation of the paragraph or text.
            Provide specific recommendations on how these graphics or infographics should be designed and what information they should convey.
            """},
            {"role": "user", "content": text}
    ]
    return callOpenAI(messages)

In [20]:
file = open('../data/extracted_data/SEA Module 3 Draft_v4', 'r')
content = file.read()
file.close()

In [ ]:
generate_graphics_idea(content)

The whole text exceeds OpenAI's token limit. Consider breaking it down into smaller sections.